In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px

directory_data = "../data"

athletes = pd.read_csv(f"{directory_data}/athlete_events.csv")
regions = pd.read_csv(f"{directory_data}/noc_regions.csv")

merged = pd.merge(athletes, regions, on="NOC", how="outer")

### Australia - histogram for ages


In [ ]:
athlete_ages = pd.read_csv(f"{directory_data}/athlete_events_fixed_age.csv")

australia_rows = athlete_ages[(athlete_ages['NOC'] == 'AUS') | (['NOC'] == 'ANZ')]

australia_rows_uniqueID = australia_rows.drop_duplicates(subset='ID') 
fig = px.histogram(australia_rows_uniqueID, x='Age', nbins=30, title="Ages of Australian athletes") 

fig.show(renderer="browser")

# make it cooler! 

### Rowing

#### Ages in rowing

In [101]:
rowing_rows = merged[merged['Sport']=='Rowing']
rowing_rows_uniqueID = rowing_rows.drop_duplicates(subset='ID')

fig = px.histogram(rowing_rows_uniqueID, x='Age', nbins=30, title="Histogram for Ages in Rowing")
fig.show(renderer="browser")

#### Medals in rowing

In [105]:
# intressant att när jag kör på Team så är både Germany och EastGermany - kolla upp

rowing_rows_medals = rowing_rows[rowing_rows['Medal'].notna()]
medals = rowing_rows_medals.groupby(['Team', 'Medal']).size().unstack(fill_value=0) #count medals per Team

medals["Total"] = medals['Gold'] + medals['Silver'] + medals['Bronze']
top10= medals.sort_values(by="Total", ascending=False).head(10).reset_index()

medal_values = ["Bronze", "Silver", "Gold"]

fig = px.bar(top10, x=medal_values, y='Team', color_discrete_map={"Gold":"#FFD700", "Silver": "#C0C0C0", "Bronze": "#CD7F32"}, title="Olympic medals in rowing per 10 top team")

fig.show(renderer="browser")